## Create Annif TSV-format Vocab from UNESCO Thesaurus

Using the current [UNESCO Thesaurus](http://vocabularies.unesco.org/browser/en/about) in RDF/XML format (i.e. `unesco-thesaurus.rdf`) [downloaded here](http://vocabularies.unesco.org/exports/thesaurus/latest/). This is a controlled and structured list of concepts used in subject analysis and retrieval of documents and publications in the fields of education, culture, natural sciences, social and human sciences, communication and information.

Although Annif does understand several formats including nt, ttl, and RDF/XML. The RDF/XML is bulky in terms of multiple labels (several languages per term) that are unnecessary in this case. Since the corpus was based on English texts, this tsv version is stripped to include the URI and English preferred label.

In [9]:
"""
Create Vocabulary Reference File From UNESCO Thesaurus

This is a script file.
"""
import os
import csv
import sys
import json
from pprint import pprint
from rdflib import Graph
from rdflib import Literal, Namespace
from rdflib.namespace import SKOS, OWL
import pandas as pd

current_directory = os.getcwd()
prj_root = os.path.dirname(current_directory)
data_dir = f'{prj_root}/data'
vocabs_dir = f'{data_dir}/vocabs'
unesco_thesauri_dir = f'{data_dir}/thesauri/unesco'
rdfxml_in = os.path.join(unesco_thesauri_dir, 'unesco-thesaurus.rdf')

UNESCO = Namespace('http://vocabularies.unesco.org/thesaurus/') 
unesco = Graph()
unesco.parse(rdfxml_in, format='xml') 
lang = 'en'

outputdupsfile ='unesco-en-duplicates.tsv'

with open(f'{vocabs_dir}/{outputdupsfile}', 'w') as vocabsfile:
    writer = csv.writer(vocabsfile, delimiter='\t', lineterminator='\n')
    for i, (subject, predicate, obj) in enumerate(unesco):
        if not (subject,predicate,obj) in unesco:
            raise Exception("Iterator / Container Protocols are Broken!!")

        try:
            if obj.language == lang:
                pLabel = unesco.preferredLabel(subject, lang=lang, default=None, labelProperties=(SKOS.prefLabel,))
                if pLabel:
                    # specify that we want preflabel not scopeNote etc
                    writer.writerow([f"<{str(subject)}>", pLabel[0][1]])
        except AttributeError as e:
            pass

In [10]:
outputfile = 'unesco-en.tsv'

vocabs_df = pd.read_csv(f'{vocabs_dir}/{outputdupsfile}', 
                   delimiter='\t', 
                   header=None,
                   na_filter=False
                  )

# remove duplicates file
os.remove(f'{vocabs_dir}/{outputdupsfile}')

# dropping ALL duplicte values based on 'URI'
# Changes are done in place otherwise need assignment to new var 
vocabs_df.drop_duplicates([0], inplace=True)

# export to nodes.tsv
processed_tsv = os.path.join(vocabs_dir, outputfile)
vocabs_df.to_csv(processed_tsv, 
                sep='\t',
                encoding='utf-8', 
                index=False,
                header=False)

vocabs_df.head(10)

,0,1
0,<http://vocabularies.unesco.org/thesaurus/conc...,Computers and development
1,<http://vocabularies.unesco.org/thesaurus/conc...,Science philosophy
2,<http://vocabularies.unesco.org/thesaurus/conc...,Agricultural libraries
3,<http://vocabularies.unesco.org/thesaurus/conc...,Arctic regions
4,<http://vocabularies.unesco.org/thesaurus/conc...,Ageing population
5,<http://vocabularies.unesco.org/thesaurus/conc...,Government educational bodies
6,<http://vocabularies.unesco.org/thesaurus/conc...,Incas
7,<http://vocabularies.unesco.org/thesaurus/conc...,Ecotourism
8,<http://vocabularies.unesco.org/thesaurus/conc...,Pop art
9,<http://vocabularies.unesco.org/thesaurus/conc...,Tungusic languages


In [17]:
#https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "asians do mot mix  { wish to comment on a letter by “citizen” which appeared in your issue of june 27.  from. the “citizen’s” letter, i have no doubt to believe that he is not a native of this country. it would be worthy for the “citizen” to note that all the institutions like schools and hospitals which the “citizen” had in mind were built long before uhuru and were strictly meant for asians. they did not dream of what we now call “uhuru”.  in those days, european community much isolated itself by having their own institutions. for this reason we had something like european hospital now known as nairobi hospital. the asians on the“other hand were expected to mix with’ us at places like’ kenyatta . national hospital, then known as king george vi hospital.  this, however, was not favourable to the asians and they resorted to building their own institutions and various schools which were never used by us until after  guru. .  asians have been known tone selfish even among themselves; for this reason, every community did something for itself. among these are chu catch gujarati hindu union~.for hindus, sir guru singh sabha for the sikh community and others, even in those days goans had some problems in finding schools for their children as they do not belong to any of these sectors.  i do-not recall a time when i have heard of an asian in a harambee day and have no idea of how the “citizen” came to learn that asians are approached for help. for one ‘thing, asians are lovers of solitude making it difficult to mix with us.  the “citizen” went out of the way by claiming that we would prefer to marry asians and yet some of their characteristics, vegetarian for instance, can not allow us to mix with them.  it is characteristic of women to beautify themselves and it is wrong to say that by straightening their hair and painting their faces, our ladies want’ to look like asians. if that is the case, asian ladies can also be said to imitate chinese by decorating their eyes.  joseph f. 0. gooda, nairobi."
doc_b = 'we feel ashamed   was ashamed, humiliated and angry fiead in the press last -week that, mi. waiboci, the director of the kenyanisa fion of personnel bureau, was reported to have claimed that some of “the so-called expatriate secretaries are not mere shorthand” typists they pre personal assistants and hold important executive responsibilities”,   so, what does that prove? that local secretaries are incapable of being anything else than “mere shorthand typists”, of being “personal assistants”. or that they can’t “hold important executive responsibili ea"  . come off it, kenyanisation bureau;- show a little more originality, and try jo get facts straight in this matter. try and   find out just bow many of our?  “citizen” secretaries are not only willing but thoroughly capable of performing. the wonders that these expatriate “personal” secretaries are supposed to be performing.  are we still in the age when it was thought that just because an article or an indivi-~ dual was “imported” » was ipso facto more valuable than a local one? do we realise that given a chance, our local. secretaries are -quite capable of proving _more than “mere shorthand typists”?  the fact is that even these highly exalted “expatriate sec _retaries’ didn’t become what  they are -overnight. they are holding those fat-salary jobs because they were given. opportunity, encouragement and fair deal by their respective “asianisation”, “americanisation” “europeanisation”, “whitenisation” or — whathave-you -nisation personnel bureau.  “our secretaries get very l{ttle..af these. opportunities. low many times do we witness  . cases where a secretary or in  “gome cases a mere. steno grapher.. is..givei job ai personal secretary to an ex-. ecutive just because she is non-black?  i assure you, mr. waiboci, that there exist some cases in some . offices where’ african secretaries are for ever doomed never to win the confidence of their. employers no matter how hard they work!  judy \m. githahu mrs,  “. jefrobt.'
doc_c = "remove mp plates  the president of kenya dissolved the national assembly to make room for a freshly elected one during the forthcoming october 14 general elections.  for this we are grateful as we shall commit our votes to elect the good and the bad, the deadwood and henchmen to be.  but even now some of the ex-mps and cabinet ministers are carrying miniature flags and mp and cm plates on their cars. havent they realised’ that they. are not mps or cms as there is no parliament for the time being?  good wine is never advertised  anyway. de “§non-political voter '  with 21 wave bands and far advanced technology, this set ranks on top of the list. 21 tuning ranges fm, am, expanded lw band for special maritime radio services and 18 shortwave bands. no wonder it receives stations from all over the world clearly and distinctly. the satellite 2000 offers considerably more ana better possibilities of reception than most high performance receivers.  superior sw unit  genuine dual conversion superheterodyne with highest image frequency rejection. rotating drum selector with 8 pre-spread, overlapping sections, for coverage of wave lengths from 60 to 10.  integrated. mains power pack with automatic recharger! ~  has an automatic recharge device tor profit accumulator 476. carrying bag 805 optional in black leatherette is available. unbelievable listening pleasure on fm  the satellite 2000 is much more than a high capacity long distance receiver for shortwave fans. it will enthral the most meticulous enthusiasts                                         features separate fm unit with separate tuning.  single sideboard reception   connection facilities for an ssb adapter are obviously mandatory for a unit  of such an intricate technical layout. - los sales & service  commercial corporation kenya limited  ~ i. box 49680. govt. rd; nairobi. tel 25778.20372020788,  . 2          switchable from 220.240 to 110.127., ac. in addition, the power pack  by sts true-to-tite ‘sound reproduction, because it has many remarkable”"
doc_d = 'this stand is wrong!  of hie, the relation between the catholic church in kenya and politics has been a subject of public debate. .  to some extent protestant churches are not open to such -debates as they seem have a well defined relation betws church and state. this. can be seen clearly in the massive support -the protestants give to freedom movements in africa.  realities and modernity are things that the catholic church has. refused to face and conservatism. seems to be very deep,. rooted. for a young country like kenya this is an unfortunate thing to have. what is. wrong with a priest. standing for a political office?  jesus himself stood. among the politicians of the time and argued with them. remarkably, his argue meets were constructive. why .  cant..our priests do the same? should they be denied what is their fundamental right — the right to stand for a public office in a democratic society? the church in kenya must not emulate rome for .there is no politics. in the vatican.  maurice cardinal tuna, in his sermon at uhuru park. castigated those priests who stood in the general election and considered some. disciplinary -measures against them. personally, i feel this stand is wrong. it ‘is impossible to split the church and politics entirely.  the church must always be 3 watchdog, the sole guardian of human rights, and’ where these are threatened, it is the duty of the church to raise the. voice of the poor in defence of justice.  - the priests by virtue of what they are, are the best people who can do this — and what. place is better than parliament itself?  "give to god what is gods and to...” jesus meant that in serving god we also had -a duty towards our country. he never at any moment meant that we cannon serve god and at the same time give our allegiance to our countries. if this were the case, then the church would find itself, at. loggerheads with the state. what catholics lack is a progressive hierarchy to accommodate the present trends. ° .  patrick jude niven-makita, _  nairobi.'
doc_e = "let us start tc think afresh  your editorial “overhaul the university” nation, december 17 should spark off fresh thinking among those who find it impossible to run progressive institutions at nairobi university and kenyatta university college. nairobi university and kenyatta. university college are not village shops whose unbusinesslike managers open or slam the doors at will. are we aware that as things stand now, kenya has  not got and university at all? the buildings, and  lecturers other staff at nairobi and kenyatta have not been serving any useful purpose since the institutions were closed half a year ago.  but some people, it “appears, have been working at the two institutions, advertising vacancies for new professors and lecturers and other staff. it would be madness to learn that any of those who applied for these posts had been ~ appointed and were waiting on the campuses for the university to re-open.  maybe some of them have earned two, three or four salaries while waiting for the lectures. what was the use of closing the institutions? has this solved any problem?  confrontation between’ the university administration and the student organisations will never solve a-single problem. is the administration so arrogant as to create a situation of umapproachability? or are the students unable to organise sensible me thuds of filing their grievances? =then—the~affairs—ofthe—univer= city are not in safe hands—which provides strong support for your call to overhaul the university. any dead wood should exit silently and find a safe place to decay in.  students at both institution’ were not expelled when the decision was taken to close them down. never was it hinted that automatic expulsion would result. to ask the students to apply for readmission as though they hed’ ever vacated their positions is absurd, to put it mildly. here is an example where people placed in positions of responsibility and authority are prone to consider too many odd things aimed at  face-saving. m. d. odinga, nairobi."

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=20)

In [18]:
from pprint import pprint
pprint(ldamodel.print_topics(num_topics=5, num_words=4), indent=2)

[ (0, '0.002*"asian" + 0.002*"church" + 0.002*"nairobi" + 0.002*"citizen"'),
  (1, '0.023*"asian" + 0.019*"secretari" + 0.015*"citizen" + 0.011*"hospit"'),
  (2, '0.031*"univers" + 0.018*"institut" + 0.014*"nairobi" + 0.014*"student"'),
  (3, '0.032*"church" + 0.015*"polit" + 0.015*"stand" + 0.015*"priest"'),
  (4, '0.013*"mp" + 0.010*"unit" + 0.010*"elect" + 0.010*"band"')]


In [20]:
doc = 'this stand is wrong!  of hie, the relation between the catholic church in kenya and politics has been a subject of public debate. .  to some extent protestant churches are not open to such -debates as they seem have a well defined relation betws church and state. this. can be seen clearly in the massive support -the protestants give to freedom movements in africa.  realities and modernity are things that the catholic church has. refused to face and conservatism. seems to be very deep,. rooted. for a young country like kenya this is an unfortunate thing to have. what is. wrong with a priest. standing for a political office?  jesus himself stood. among the politicians of the time and argued with them. remarkably, his argue meets were constructive. why .  cant..our priests do the same? should they be denied what is their fundamental right — the right to stand for a public office in a democratic society? the church in kenya must not emulate rome for .there is no politics. in the vatican.  maurice cardinal tuna, in his sermon at uhuru park. castigated those priests who stood in the general election and considered some. disciplinary -measures against them. personally, i feel this stand is wrong. it ‘is impossible to split the church and politics entirely.  the church must always be 3 watchdog, the sole guardian of human rights, and’ where these are threatened, it is the duty of the church to raise the. voice of the poor in defence of justice.  - the priests by virtue of what they are, are the best people who can do this — and what. place is better than parliament itself?  "give to god what is gods and to...” jesus meant that in serving god we also had -a duty towards our country. he never at any moment meant that we cannon serve god and at the same time give our allegiance to our countries. if this were the case, then the church would find itself, at. loggerheads with the state. what catholics lack is a progressive hierarchy to accommodate the present trends. ° .  patrick jude niven-makita, _  nairobi.'
corpus_1= dic_1.doc2bow(tokenize(doc))

new_predictions= ldamodel[corpus_1]

NameError: name 'dic_1' is not defined

In [27]:
import pandas as pd 
train=pd.DataFrame({'text':['find the most representative document for each topic',
                        'topic distribution across documents',
                        'to help with understanding the topic',
                        'one of the practical application of topic modeling is to determine']})
text=pd.DataFrame({'text':['how to find the optimal number of topics for topic modeling']})


def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#using your train data to train the model with 4 topics

data_words = list(sent_to_words(train['text']))
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=4)

pprint(lda_model.print_topics(num_topics=5, num_words=4), indent=2)

#  predicting new text which is in text dataframe  
new_text_corpus =  id2word.doc2bow(text['text'][0].split())
lda_model[new_text_corpus]

#op

#Out[75]:
# [(0, 0.5517368), (1, 0.38150477), (2, 0.032756805), (3, 0.03400166)]

[ (0, '0.047*"topic" + 0.046*"the" + 0.046*"distribution" + 0.046*"to"'),
  (1, '0.047*"topic" + 0.046*"the" + 0.046*"distribution" + 0.046*"documents"'),
  (2, '0.092*"topic" + 0.092*"of" + 0.092*"the" + 0.051*"to"'),
  (3, '0.145*"topic" + 0.081*"the" + 0.081*"with" + 0.081*"to"')]


[(0, 0.031387348), (1, 0.031386077), (2, 0.9046622), (3, 0.032564398)]